In [47]:
import os
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
from nltk.stem.snowball import SnowballStemmer
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report, accuracy_score
import seaborn
from sklearn.svm import SVC
from sklearn.feature_extraction.text import TfidfVectorizer
#nltk.download('all')

In [2]:
os.environ['kaggle_data'] = '/Users/mac/Downloads/ML_practise/NLP exercise/Sentiment_analysis'

In [3]:
train_data = 'data/train.tsv.zip'
test_data = 'data/test.tsv.zip'
label = 'data/sampleSubmission.csv'

In [26]:
df_train = pd.read_csv(train_data, sep ='\t')
df_train.head()

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,1
1,2,1,A series of escapades demonstrating the adage ...,2
2,3,1,A series,2
3,4,1,A,2
4,5,1,series,2


In [14]:
df_test = pd.read_csv(test_data, sep='\t')
df_test.head()

,PhraseId,SentenceId,Phrase
0,156061,8545,An intermittently pleasing but mostly routine ...
1,156062,8545,An intermittently pleasing but mostly routine ...
2,156063,8545,An
3,156064,8545,intermittently pleasing but mostly routine effort
4,156065,8545,intermittently pleasing but mostly routine


In [6]:
def preprocess(doc):
    #token
    #remove stopwords
    #lemmization
    #return join
    token = word_tokenize(doc.lower())
    tokenz=[]
    for tok in token:
        if tok not in stopwords.words('english'):
            tokenz.append(tok)
    vector = WordNetLemmatizer()
    data = [vector.lemmatize(tok) for tok in tokenz]
    return data

In [21]:
#df_train['Phrase']= df_train.Phrase.apply(preprocess)
#print(df_train.Phrase.apply(preprocess))

In [8]:
#df_test['Phrase']= df_test.Phrase.apply(preprocess)

In [23]:
stemmer = SnowballStemmer('english')

In [24]:
def tokenizeWithoutPunctuation(text):
    return [stemmer.stem(token) for token in word_tokenize(text) if token.isalpha()]

In [18]:
s_w = stopwords.words('english')
stop_ = s_w[:115]

In [28]:
tfidf = TfidfVectorizer(tokenizer=tokenizeWithoutPunctuation, stop_words=stop_, ngram_range=(1,2), max_features=2000)
vectors = tfidf.fit_transform(df_train['Phrase'])

In [30]:
clf = SVC()
clf.fit(vectors, df_train['Sentiment'])

SVC()

In [41]:
train = vectors[0:2000,:]
train.shape

(2000, 2000)

In [44]:
y_train = df_train['Sentiment'][0:2000]
y_train

0       1
1       2
2       2
3       2
4       2
       ..
1995    2
1996    1
1997    2
1998    2
1999    2
Name: Sentiment, Length: 2000, dtype: int64

In [31]:
test = tfidf.transform(df_test['Phrase'])

In [32]:
test.shape

(66292, 2000)

In [33]:
test = test[1:200,:]

In [45]:
predict = clf.predict(train)

In [48]:
accuracy_score(y_train, predict)

0.7795

In [50]:
print(classification_report(y_train, predict))

              precision    recall  f1-score   support

           0       0.90      0.19      0.32        47
           1       0.73      0.50      0.59       294
           2       0.80      0.95      0.87      1251
           3       0.67      0.56      0.61       316
           4       0.89      0.36      0.51        92

    accuracy                           0.78      2000
   macro avg       0.80      0.51      0.58      2000
weighted avg       0.78      0.78      0.76      2000

